In [ ]:
from pathlib import Path
from datetime import datetime, timezone
import pandas as pd
import sqlite3

ROOT = Path(__file__).resolve().parents[1]
OUT = ROOT / "output"
PARQUET_DIR = OUT / "parquet"
QUALITY_DIR = OUT / "quality"
PARQUET_DIR.mkdir(parents=True, exist_ok=True)
QUALITY_DIR.mkdir(parents=True, exist_ok=True)
DB = OUT / "ut1.db"
con = sqlite3.connect(DB)
# Si tienes 00_schema.sql lo puedes ejecutar antes (opcional)
con.executescript((ROOT / "sql" / "00_schema.sql").read_text(encoding="utf-8"))
# ingesta  
def to_float_money(x):
    try:
        return float(str(x).replace(",", "."))
    except Exception:
        return None

def strip_strings(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.str.strip()
    for c in df.columns:
        if pd.api.types.is_object_dtype(df[c]):
            df[c] = df[c].astype(str).str.strip()
    return df


from pathlib import Path
from datetime import datetime, timezone
import pandas as pd
import sqlite3

ROOT = Path(__file__).resolve().parents[1]
DATA = ROOT / "data" / "drops"
OUT = ROOT / "output"
PARQUET_DIR = OUT / "parquet"
QUALITY_DIR = OUT / "quality"
OUT.mkdir(parents=True, exist_ok=True)
PARQUET_DIR.mkdir(parents=True, exist_ok=True)
QUALITY_DIR.mkdir(parents=True, exist_ok=True)

def to_float_money(x):
    try:
        return float(str(x).replace(",", "."))
    except Exception:
        return None

def strip_strings(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.str.strip()
    for c in df.columns:
        if pd.api.types.is_object_dtype(df[c]):
            df[c] = df[c].astype(str).str.strip()
    return df

def ingest_one(f: Path) -> pd.DataFrame:
    if f.suffix.lower() == ".csv":
        df = pd.read_csv(f, dtype=str)
    elif f.suffix.lower() in {".ndjson", ".jsonl"}:
        df = pd.read_json(f, lines=True, dtype=str)
    else:
        raise ValueError(f"Formato no soportado: {f.suffix}")
    df = strip_strings(df)
    if "fecha_venta" in df.columns:  # ventas.csv
        df = df.rename(columns={"fecha_venta": "fecha"})
    df["_source_file"] = f.name
    df["_ingest_ts"] = datetime.now(timezone.utc).isoformat()
    return df

def ingest_all_csvs_to_raw(con: sqlite3.Connection) -> dict:
    """
    Ingresa todos los .csv de data/drops a tablas RAW:
    - raw_ventas, raw_clientes, raw_productos
    Retorna contadores por tipo.
    """
    counters = {"ventas": 0, "clientes": 0, "productos": 0}
    for f in sorted(DATA.glob("*.csv")):
        df = ingest_one(f)
        stem = f.stem.lower()
        # Ruteo por nombre de archivo
        if "ventas" in f.name.lower():
            # Esperado: fecha (renombrada), id_cliente, id_producto, unidades, precio_unitario
            needed = ["fecha", "id_cliente", "id_producto", "unidades", "precio_unitario", "_ingest_ts", "_source_file"]
            for c in needed:
                if c not in df.columns:
                    df[c] = None
            df_raw = df[needed].copy()
            df_raw["_batch_id"] = stem
            df_raw.to_sql("raw_ventas", con, if_exists="append", index=False)
            counters["ventas"] += len(df_raw)
        elif "clientes" in f.name.lower():
            # Esperado: fecha, nombre, apellido, id_cliente
            cols = ["fecha", "nombre", "apellido", "id_cliente"]
            for c in cols:
                if c not in df.columns:
                    df[c] = None
            df_raw = df[cols + ["_ingest_ts", "_source_file"]].copy()
            df_raw["_batch_id"] = stem
            df_raw.to_sql("raw_clientes", con, if_exists="append", index=False)
            counters["clientes"] += len(df_raw)
        elif "productos" in f.name.lower():
            # Esperado: fecha_entrada, nombre_producto, id_producto, unidades, precio_unitario, categoria
            cols = ["fecha_entrada", "nombre_producto", "id_producto", "unidades", "precio_unitario", "categoria"]
            for c in cols:
                if c not in df.columns:
                    df[c] = None
            df_raw = df[cols + ["_ingest_ts", "_source_file"]].copy()
            df_raw["_batch_id"] = stem
            df_raw.to_sql("raw_productos", con, if_exists="append", index=False)
            counters["productos"] += len(df_raw)
        else:
            # Otros CSV: si aparecen, los puedes ignorar o enrutar a una raw_otros
            pass
    return counters
